In [2]:
import os
import datetime
import calendar
import pandas as pd


ENTRY_NAMES = ['work', 'Work', 'Bar']  # Names used

workcal_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
rel_path = 'data/workfiles'
workfiles_path = os.path.join(workcal_path, rel_path)
filelist = os.listdir(workfiles_path)  # makes pathes independent of containing superfolder

worktimes_all = []  # stores all for all days the worktimes of a day in list

# todo: search all *.ics files via os
def delta2min(delta):
    return divmod(delta.seconds, 60)[0]

for filename in filelist:
    try:
        with open(os.path.join(workfiles_path, filename), 'r') as file:
            content = file.read()
            for possible_entry in ENTRY_NAMES:
                if possible_entry in content:
                    filelines = content.split('\n')
                    worktimes_all.append([x for x in filelines if 'Europe/Berlin:' in x])
    except:
        pass
# ['DTEND;TZID=Europe/Berlin:20191031T220000', 'DTSTART;TZID=Europe/Berlin:20191031T170000']

def parse(entry):
    kind, dtime = entry.split('Europe/Berlin:')
    dtime = datetime.datetime.strptime(dtime, '%Y%m%dT%H%M%S')
    kind = 'start' if 'start'.upper() in kind else 'end'
    return kind,dtime


worktimes_parsed = [[parse(entry)for entry in day]for day in worktimes_all if len(day) == 2]
workrange_shift = [(b[1] if a[0] == 'end' else a[1], a[1]-b[1] if a[0] == 'end' else b[1]-a[1]) for a, b in worktimes_parsed]
workrange_shift = [(dtime, delta2min(shiftrange)) for dtime,shiftrange in workrange_shift]
workrange_shift.sort()

df = pd.DataFrame(workrange_shift,columns=['datetime_shift','amount_min'])
df

,datetime_shift,amount_min
0,2019-01-18 20:30:00,30
1,2019-09-21 10:30:00,240
2,2019-09-30 17:00:00,300
3,2019-10-01 17:00:00,300
4,2019-10-02 11:00:00,210
5,2019-10-03 11:00:00,210
6,2019-10-04 12:00:00,210
7,2019-10-04 17:00:00,300
8,2019-10-05 11:00:00,210
9,2019-10-18 17:00:00,330


In [3]:
import datetime
from datetime import date

dtime_amountminutes = workrange_shift
tday = date.today()
endoflastmon = tday - datetime.timedelta(days=tday.day)
startoflastmon = endoflastmon - datetime.timedelta(days=endoflastmon.day-1)
workedlastmonth = [ws for ws in dtime_amountminutes if startoflastmon <= ws[0].date() <= endoflastmon]

In [4]:
df = pd.DataFrame([list(entry) + 2*[None] for entry in workedlastmonth],columns=['datetime_shiftstart','Arbeitszeit','Pausenzeiten von','Pausenzeiten bis'])
df

df['Arbeitsbeginn'] = [f'{dt.hour}:{dt.minute if dt.minute != 0 else "00"}' for dt in df['datetime_shiftstart']]
df['Tag'] = [dt.date().day for dt in df['datetime_shiftstart']]
df['Arbeitszeit'] = df['Arbeitszeit'].apply(lambda x: f'{x//60 if x//60 != 0 else "00"}:{x%60 if x%60 != 0 else "00"}')
df['Arbeitsende'] = df['Arbeitsbeginn'] + ',' + df['Arbeitszeit']
df['Arbeitsende'] = df['Arbeitsende'].apply(lambda x: [[ int(hm) for hm in t.split(':')] for t in  x.split(',')] )
df['Arbeitsende'] = df['Arbeitsende'].apply(lambda x: f'{x[0][0] + x[1][0]}:{x[0][1] + x[1][1] if x[0][1] + x[1][1] != 0 else "00"}')

split_shift = list(set([ day for day in df['Tag']if list(df['Tag']).count(day) > 1]))

def add_worktime_splitshift(x):
    return f'{x[0]+x[2]}:{(x[1]+x[3])%60 if (x[1]+x[3])%60 != 0 else "00"}'

drops = []
for i in range(len(df) - 1):
    if df.loc[i,'Tag'] == df.loc[i+1,'Tag']:
        df.loc[i,'Pausenzeiten von'] = df.loc[i,'Arbeitsende']
        df.loc[i,'Pausenzeiten bis'] = df.loc[i+1,'Arbeitsbeginn']
        df.loc[i,'Arbeitsende'] = df.loc[i+1,'Arbeitsende']
        df.loc[i,'Arbeitszeit'] = add_worktime_splitshift([int(hm) for hm in df.loc[i,'Arbeitszeit'].split(':')] + [int(hm) for hm in df.loc[i+1,'Arbeitszeit'].split(':')])
        drops.append(i+1)
for row in drops:
    df = df.drop(row)
    
    
df = df.drop('datetime_shiftstart',axis=1)
df['Entlohnungsart'] = len(df)*[None]

c = df.columns.tolist()
c[0], c[1], c[2], c[3], c[4], c[5], c[6]  = c[3], c[5], c[1], c[2], c[0], c[6], c[4]
df = df[c]
df.index += 1
df.index[df['Tag'] == 4].tolist()

[]

In [5]:
# blank for month
blank = [None]*6
real = pd.DataFrame([blank]*31,columns=['Arbeitsbeginn','Arbeitsende','Pausenzeiten von','Pausenzeiten bis','Arbeitszeit','Entlohnungsart'])
real.index += 1 

for i in range(1,32):
    isday = df[df['Tag'] == i]
    isday = isday.drop('Tag',axis=1)
    if not isday.empty:
        real.loc[i,:] = isday.iloc[0]
        
real.index.name = 'Tag'
real

,Arbeitsbeginn,Arbeitsende,Pausenzeiten von,Pausenzeiten bis,Arbeitszeit,Entlohnungsart
Tag,,,,,,
1,17:00,22:00,None,None,5:00,None
2,17:00,22:00,None,None,5:00,None
3,11:00,14:00,None,None,3:00,None
4,None,None,None,None,None,None
5,None,None,None,None,None,None
6,None,None,None,None,None,None
7,None,None,None,None,None,None
8,None,None,None,None,None,None
9,None,None,None,None,None,None


In [14]:
from PyPDF2 import PdfFileReader
import tabula

fpath = os.path.join(workcal_path, 'data/stundenaufzeichnung_mindestlohn.pdf')
with open(fpath, 'rb') as f:
    pdf = PdfFileReader(f)
    info = pdf.getDocumentInfo()

info


{'/SourceModified': 'D:20150102081434',
 '/Author': 'ADDISON Akadmie',
 '/Subject': 'MiLoG',
 '/Title': 'Stundenaufzeichnung',
 '/CreationDate': "D:20150102091510+01'00'",
 '/ModDate': "D:20191003232132Z00'00'",
 '/Creator': 'Acrobat PDFMaker 10.1 für Word',
 '/Producer': 'iOS Version 13.1.2 (Build 17A860) Quartz PDFContext, AppendMode 1.1',
 '/Company': 'ADDISON Software und Service GmbH'}